In [ ]:
%%capture
!pip install langchain openai cohere huggingface_hub transformers accelerate

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key:··········


In [ ]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

Cohere API Key:··········


In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("HF API Key:")

HF API Key:··········


# Model I/O

There are three components to this module:

  1) Prompts, which provide templates that allow you to parametrize and reuse prompts.

  2) Language models, which allow you to interface with:

  • LLMs, which take a string as input and return a string as output
  
  • Chat models, which take a list of chat messages as input and return

  3) Output Parsers allow you to extract and structure the text output from a language model in the way you want. These are useful for tasks like QA, where you must parse an answer.

The typical workflow is as follows:

• Choose either an LLM or a Chat model; this depends on your use case

• Construct a prompt that you customize as the inputs

• Send the input to the LLM or Chat model

• Parse outputs with output parsers, if needed

## 🖱️ Let's double-click on the language models component.  

In general, all Chat models are LLMs. But, not all LLMs are Chat models.

But both implement the same Base Language Model interface, making it easy to swap between them.

### 🗣️ LLMs:

• Take a string as input and return a string as output

• Implemented for models optimized for text completion

• Expose a `predict` method that takes a string prompt and returns a string completion

### 💬 Chat models:

• Take a list of `ChatMessages` as input and return a `ChatMessage` as output

• Implemented for models that are tuned for dialogue

• Expose a `predict_messages` method that takes a list of ChatMessages and returns a single ChatMessage

• ChatMessages contain a content field with the message text and a role field indicating the speaker

### 🔑 The key differences:
• Input/output types: strings vs. ChatMessages

• Methods: `predict` vs. `predict_messages`

• Optimization: text completion vs. dialog

# Working LLMs

The LLM class is designed to be a standard interface to an LLM provider. This class abstracts away provider-specific APIs and exposes common methods like `predict` and `generate`.

• `predict` is optimized for text completion. It allows you to format a prompt template and pass it to the language model. The output is just plain text.

• `generate` takes a list of prompts and returns detailed `LLMResult` objects with completions and metadata.

Let's instantiate LLMs from Cohere, OpenAI, and HuggingFace and take a look at the difference between the generate and predict method.

## Predict method

Notice that it's the same API for each of the different models, which is nice.


In [ ]:
from langchain.llms import OpenAI, Cohere, HuggingFacePipeline, HuggingFaceHub
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import transformers
import torch

openai_llm = OpenAI()

cohere_llm = Cohere()

model_id = 'Deci/DeciLM-6b-instruct'

task = "text-generation"

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map="auto",
                                             trust_remote_code=True
                                             )

tokenizer = AutoTokenizer.from_pretrained(model_id)

pipe = pipeline(
    task,
    model=model,
    tokenizer=tokenizer,
    temperature=0,
    max_new_tokens=512,
    num_beams=1,
    early_stopping=True,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

huggingface_llm = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
prompt = "How do I become an AI Engineer?"

In [ ]:
openai_llm.predict(prompt)

"\n\n1. Earn a bachelor's degree in computer science or a related field. Most AI engineers possess a degree in computer science, software engineering, or a related field.\n\n2. Develop a strong background in mathematics and programming. AI engineering requires an extensive knowledge of mathematics and programming languages such as Python and Java.\n\n3. Gain experience with AI technologies. AI engineers must be familiar with AI technologies such as machine learning, natural language processing, and robotics. You should also become familiar with AI frameworks such as TensorFlow and Apache Spark.\n\n4. Build a portfolio. You should create a portfolio of projects that demonstrate your skills in AI engineering.\n\n5. Consider certification. You may consider obtaining an AI certification to demonstrate your knowledge and skills."

In [ ]:
cohere_llm.predict(prompt)

" There is no single path to becoming an AI engineer, but there are several steps you can take to get started in the field. First, it is important to have a strong foundation in computer science and mathematics, as these are the building blocks of AI. You should also be familiar with programming languages such as Python, R, and Java, as well as data structures and algorithms. \n\nIn addition, it is helpful to have experience with machine learning algorithms and deep learning frameworks. You should also be familiar with data structures and algorithms, as well as statistics and probability theory. \n\nFinally, it is important to have strong problem-solving and analytical skills, as well as the ability to work in a team. You should also be able to communicate complex ideas clearly and concisely. \n\nIf you have these skills and experience, you may be ready to pursue a career as an AI engineer. There are several ways to do this, including earning a master's or PhD in computer science or a 

In [ ]:
huggingface_llm.predict(prompt)

### Generate

And as you can see below, the generate method gives more details

In [ ]:
# when using .generate you need to pass in a list
openai_llm.generate([prompt])

LLMResult(generations=[[Generation(text='\n\nHarpreet Sahota, the legendary data scientist, was born and raised in Sacramento, CA. She is now making her mark in the data science world, inspiring many in her hometown.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'completion_tokens': 40, 'total_tokens': 66, 'prompt_tokens': 26}, 'model_name': 'text-davinci-003'}, run=[RunInfo(run_id=UUID('429f6d3f-37c3-43a4-a7f4-e1b829690d9a'))])

# Compare model outputs

In [ ]:
from langchain.model_laboratory import ModelLaboratory

model_lab = ModelLaboratory.from_llms([openai_llm, cohere_llm, huggingface_llm])

model_lab.compare(prompt)

Input:
How do I become an AI Engineer?

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'max_tokens': 256, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


1. Obtain a bachelor’s degree in computer science, software engineering, mathematics, physics, or another related field.

2. Gain experience with AI-related technologies, such as machine learning and deep learning.

3. Become proficient in coding languages, such as Python, C++, and R.

4. Build a portfolio of projects that demonstrate your AI engineering skills.

5. Pursue a master’s degree in AI engineering or a related field.

6. Familiarize yourself with industry trends and developments in AI technology.

7. Consider obtaining certifications or attending workshops that can help you develop the necessary skills.

8. Network with other AI engineers and stay up-to-date on the latest advancements in the field.

Cohere
Params: {'model': None, 'max_tok

# Working with Chat models

We'll stick to the OpenAI chat models for this section.

The chat model interface is based around messages rather than raw text.

The types of messages currently supported in LangChain are `AIMessage`, `HumanMessage`, `SystemMessage`.

In [ ]:
from langchain.chat_models import ChatOpenAI

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

llm = OpenAI(model_name="gpt-3.5-turbo")
chat = ChatOpenAI()

messages = [
    SystemMessage(content="You are a tough love career coach who gets to the point and pushes your mentees to be their best."),
    HumanMessage(content="How do I become an AI engineer?")
]

chat(messages)

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


AIMessage(content="Becoming an AI engineer requires dedication, hard work, and a solid skill set. Here's a roadmap to get you started:\n\n1. Education: Pursue a degree in computer science, data science, or a related field. Get a strong foundation in mathematics, statistics, and programming languages like Python.\n\n2. Specialize: Focus on artificial intelligence, machine learning, and deep learning. Take advanced courses in these areas and gain a deep understanding of algorithms, data structures, and optimization techniques.\n\n3. Projects: Build a portfolio of AI projects to showcase your skills. Start with small projects and gradually work on more complex ones. Collaborate with others to gain experience in real-world scenarios.\n\n4. Practical Experience: Look for internships or entry-level positions in companies that work on AI projects. Gain hands-on experience, learn from experts, and apply your skills to solve real-world problems.\n\n5. Continuous Learning: AI is an ever-evolving

## Prompts:

1. **Language models (LLMs)** require prompts to function.

2. A prompt is a set of instructions or input provided to guide the model's response.

3. The output from a prompt can be answers, sentence completions, or conversation responses.

4. A well-constructed prompt template has the following parts:
   - **Instructions**: Define the model's response/behavior.
   - **Context**: Provides additional information, sometimes with examples.
   - **User Input**: The actual question or input from the user.
   - **Output Indicator**: Marks the beginning of the model's response.


## What is a Prompt Template?

A `PromptTemplate` allows creating a template string with placeholders, like `{adjective}` or `{content}` that can be formatted with input values to create the final prompt string.

It's a reproducible way to generate, share, and reuse prompts.

Contains:
   - A text string (template) that takes inputs and produces a prompt for the LLM.
   - Instructions for the LLM.
   - Few-shot examples to enhance the model's response.
   - A question for the language model.

In [ ]:
from langchain import PromptTemplate, OpenAI

# Define a simple prompt template as a Python string

prompt_template = PromptTemplate.from_template("""
Human: What is the capital of {place}?
AI: The capital of {place} is {capital}
""")

prompt = prompt_template.format(place="California", capital="Sacramento")

print(prompt)


Human: What is the capital of California?
AI: The capital of California is Sacramento



/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:24: UserWarning: Importing PromptTemplate from langchain root module is no longer supported.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:24: UserWarning: Importing OpenAI from langchain root module is no longer supported.
  warnings.warn(


In [ ]:
# No Input Variable
no_input_prompt = PromptTemplate(input_variables=[], template="Tell me a joke.")
print(no_input_prompt.format())

# One Input Variable
one_input_prompt = PromptTemplate(input_variables=["adjective"], template="Tell me a {adjective} joke.")
print(one_input_prompt.format(adjective="funny"))

# Multiple Input Variables
multiple_input_prompt = PromptTemplate(
 input_variables=["adjective", "content"],
 template="Tell me a {adjective} joke about {content}."
)

multiple_input_prompt = multiple_input_prompt.format(adjective="funny", content="chickens")
print(multiple_input_prompt)

Tell me a joke.
Tell me a funny joke.
Tell me a funny joke about chickens.


Pass a prompt template to an LLM

In [ ]:
llm = OpenAI()

prompt_template = PromptTemplate.from_template(
    template="Write a {length} story about: {content}"
)

prompt = prompt_template.format(
    length="2-sentence",
    content="Sacramento, CA, the hometown of the legendary data scientist, Harpreet Sahota"
)

response = llm.predict(
    text=prompt
)

print(response)



When Harpreet Sahota was growing up in Sacramento, CA, little did he know that he would one day become a legendary data scientist. Now, his hometown is proud to have him as one of their own.


# Output parsers

Depending on the downstream uses, raw text from a language model might not be what you need.

Output parsers are classes in Langchain that help structure the text responses from language models into more useful formats. Output parsers allow you to convert the text into JSON, Python dataclasses, database rows, and more.

## What are they used for?

Output parsers have two main uses:

1) Convert unstructured text into structured data. For example, parsing text into a JSON or Python object.

2) Inject instructions into prompts to tell language models how to format their responses. The parser can provide a `get_format_instructions()` method that returns text for the prompt.

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

from langchain.output_parsers.list import ListOutputParser

Without parsing output

In [ ]:
llm = OpenAI()

prompt = PromptTemplate(
    template="List 3 {things}",
    input_variables=["things"])

llm.predict(text=prompt.format(things="sports that don't use balls"))

'\n\n1. Swimming \n2. Running \n3. Rock Climbing'

Instantiate output parser

In [ ]:
output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz`


Now let's see how to use the parsers instructions in the prompt

In [ ]:
prompt = PromptTemplate(
    template="List 3 {things}.\n{format_instructions}",
    input_variables=["things"],
    partial_variables={"format_instructions": format_instructions})

output = llm.predict(text=prompt.format(things="sports that don't use balls"))

print(output)



Cricket, Golf, Archery


Finally, we can parse the output to a list (Python object)

In [ ]:
output_parser.parse(output)

['Cricket', 'Golf', 'Archery']